# Analysis of the astrocytes RNA-seq data from HPT, CTX, and Hippo  

## Table of contents:

* <a href=#Load>Load packages and set global variables</a>
* <a href=#Dataloading>Loading data, preprocessing and make DEseq object</a>
* <a href=#Visualisataion>Visualisation of the data</a>
* <a href=#DGE>Differential gene expression - diet effect</a>
* <a href=#Celltype>Heatmap marker genes</a>
* <a href=#HeatmapDGE>Heatmap DGE</a>
* <a href=#GOenrichment>GO pathway enrichment</a>

## Load packages and set global variables

In [1]:
inst = suppressMessages(lapply(c('DESeq2',
                                 'vidger',
                                 'pheatmap',
                                 'ggplot2',
                                 'EnhancedVolcano',
                                 'VennDiagram',
                                 'org.Mm.eg.db',
                                 'clusterProfiler',
                                 'UpSetR',
                                 'readxl',
                                 'tidyverse',
                                 'KEGGREST'),
                               library,
                               character.only=TRUE)
) 

In [2]:
prot_input_file = '~/Documents/consultation/Luiza/omics_data/data/astrocyteprotoeme12102017-1.xlsx'
options(repr.plot.width=8, repr.plot.height=8)
my_palette = colorRampPalette(c("blue", "white", "red"))(n=255)

Set whether anndata objects are recomputed or loaded.

In [3]:
bool_recomp = FALSE

Set whether to produce plots, set to False for test runs.

In [4]:
bool_plot = FALSE

## Loading data, preprocessing and make DEseq object 

In [5]:
dat=read.csv(file="/Users/viktorian.miok/Documents/data/RNA-seq_Data-NEW/new_counts1.csv", 
             header=TRUE,
             sep=",")
rownames(dat)=dat[,1]
dat=dat[,-1]

In [6]:
if(bool_recomp){  
    # make a meta data
    group=c(rep("HPT_Chow",6),rep("HPT_HFD",7),rep("Hippo_Chow",6),rep("Hippo_HFD",7),rep("CTX_Chow",6),rep("CTX_HFD",7))
    lab=colnames(dat)
    loc=c(rep("Hypothalamus",13), rep("Hippocampus",13), rep("Cortex",13))
    diet=c(rep("chow",6), rep("hfd",7), rep("chow",6), rep("hfd",7), rep("chow",6), rep("hfd",7))
    id=colnames(dat)
    metaData=data.frame(id, group, loc,diet)

    # remove probleatic samples
    dat=dat[,-c(10,13,19,34)] #dat[,-c(10,13,19,34,36)]    19,,36  #10,13,19,32,39
    metaData=metaData[-c(10,13,19,34),] #metaData[-c(10,13,19,34,36),]    19,36

    # convert from ensembl to symbol gene id
    rownames(dat)=gsub("\\..*","",rownames(dat))
    sig.gene=bitr(gsub("\\..*","",rownames(dat)), fromType="ENSEMBL", toType="SYMBOL", OrgDb=org.Mm.eg.db)
    symb=sig.gene$SYMBOL[ match(rownames(dat), sig.gene$ENSEMBL) ]
    dat=dat[!is.na(symb),]
    symb=symb[!is.na(symb)]
    rownames(dat)=make.names(symb,TRUE)

    # filtering
    keep=rowSums(dat) > 10
    dat=dat[keep,]

    keep=rowSums(dat >= 10) >= 5
    dat=dat[keep,]

    #make a deseq2 object
    dds=DESeqDataSetFromMatrix(countData=dat, colData = metaData, design=~ group)
    dds=DESeq(dds)
    save(dds, file="dds.RData")
} else {
    load("dds.RData")
}

## Visualisation of the data

In [7]:
# variance stabilizing transformation data
vsd = assay(vst(dds, blind=TRUE))

In [8]:
pdf(file="SM/PCA_hpt.pdf", width=10, height=10)
    pcaData = plotPCA(vst(dds[,1:11]),
                       intgroup=c("group"),
                       returnData=TRUE
    )
    gr1 = dds@colData$group[1:11]
    percentVar = round(100 * attr(pcaData, "percentVar"))

    ggplot(pcaData, aes(x=PC1, y=PC2, color=gr1)) +
           geom_point(aes(color=gr1), size=5) + 
           geom_point(shape=1, color="black", size=5) + 
           xlab(paste0("PC1: ",percentVar[1],"% variance")) + 
           ylab(paste0("PC2: ",percentVar[2],"% variance")) +
           theme(axis.text=element_text(size=20), 
                 axis.title=element_text(size=20), 
                 legend.title=element_text(siz=20),
                 legend.text=element_text(size=20),
                 plot.title=element_text(size=20)) + 
           scale_color_manual(values=c("chocolate","tan1")) +
           ylim(-25,15) + xlim(-20,20)
dev.off()

pdf 
  2

In [9]:
pdf(file="SM/PCA_hippo.pdf", width=10, height=10)
    pcaData = plotPCA(vst(dds[,12:23]),
                       intgroup=c("group"),
                       returnData=TRUE
    )
    gr1 = dds@colData$group[12:23]
    percentVar = round(100 * attr(pcaData, "percentVar"))

    p = ggplot(pcaData, aes(x=PC1, y=PC2, color=gr1)) +
                geom_point(aes(color=gr1), size=5) + 
                geom_point(shape = 1, color = "black", size = 5) + 
                xlab(paste0("PC1: ",percentVar[1], "% variance")) + 
                ylab(paste0("PC2: ",percentVar[2], "% variance")) +
                theme(axis.text = element_text(size=20), 
                      axis.title = element_text(size=20), 
                      legend.title = element_text(siz=20),
                      legend.text = element_text(size=20),
                      plot.title = element_text(size=20)) + 
                scale_color_manual(values = c("mediumseagreen", "lightgreen"))
dev.off()

pdf 
  2

In [10]:
pdf(file="SM/PCA_ctx.pdf", width=10, height=10)
    pcaData = plotPCA(vst(dds[,24:35]),
                       intgroup=c("group"),
                       returnData=TRUE
    )
    gr1 = dds@colData$group[24:35]
    percentVar = round(100 * attr(pcaData, "percentVar"))

    ggplot(pcaData, aes(x=PC1, y=PC2, color=gr1)) +
           geom_point(aes(color=gr1), size=5) + 
           geom_point(shape=1, color="black", size=5) + 
           xlab(paste0("PC1: ",percentVar[1], "% variance")) + 
           ylab(paste0("PC2: ",percentVar[2], "% variance")) +
           theme(axis.text=element_text(size=20), 
                 axis.title=element_text(size=20), 
                 legend.title=element_text(siz=20),
                 legend.text=element_text(size=20),
                 plot.title=element_text(size=20)) + 
           scale_color_manual(values=c("lightblue4", "lightblue"))
dev.off()

pdf 
  2

In [11]:
if(bool_plot){ 
    df_pca = prcomp(t(vsd))

    df_out = as.data.frame(df_pca$x)
    df_out$tissue = as.factor(dds@colData$group) 

    ggplot(df_out, aes(x=PC1, y=PC2, color=tissue)) + 
           geom_point(aes(color=tissue), size=5) + 
           geom_point(shape=1, color="black", size=5) + 
           xlab(paste0("PC1: ",round(df_pca$sdev[1]), "% variance")) + 
           ylab(paste0("PC2: ",round(df_pca$sdev[2]), "% variance")) +
           theme(axis.text=element_text(size=20), 
                 axis.title=element_text(size=20), 
                 legend.title=element_text(siz=20),
                 legend.text=element_text(size=20),
                 plot.title=element_text(size=20)) + 
           scale_color_manual(values=c("lightblue", "lightblue4", "tan1",
                                       "chocolate", "lightgreen", "mediumseagreen"))
}

In [12]:
pdf(file="PCA.pdf", width=10, height=10)
    pcaData = plotPCA(vst(dds),
                       intgroup=c("group"),
                       returnData=TRUE
    )
    percentVar = round(100 * attr(pcaData, "percentVar"))

    ggplot(pcaData, aes(x=PC1, y=PC2, color=dds@colData$group)) +
           geom_point(aes(color=dds@colData$group), size=10) + 
           geom_point(shape=1, color="black", size=10) + 
           xlab(paste0("PC1: ",percentVar[1],"% variance")) + 
           ylab(paste0("PC2: ",percentVar[2],"% variance")) +
           theme(axis.text=element_text(size=25), 
                 axis.title=element_text(size=30)) +
           scale_color_manual(values=c("aquamarine4", "aquamarine2", "darkorange2",
                                       "orange", "blueviolet", "mediumpurple1")) +
           theme(legend.position="none")
dev.off()

pdf 
  2

In [13]:
# Heatmap of all samples and genes
pdf(file="SM/heatmap_alldat.pdf", width=10, height=10)
    agsh = pheatmap(vsd,
                    cluster_rows=T, 
                    scale="row", 
                    show_rownames=FALSE, 
                    color=my_palette,
                    cluster_cols=F, 
                    annotation_col=as.data.frame(colData(dds)[,c("diet", "loc")]),
                    fontsize=10,
                    breaks=seq(-3, 3, length.out=255)
    )
dev.off()

pdf 
  3

## Differential gene expression - diet effect

In [14]:
if(bool_recomp){ 
    hpt = results(dds, contrast=c("group", "HPT_HFD", "HPT_Chow"))
    hippo = results(dds, contrast=c("group", "Hippo_HFD", "Hippo_Chow"))
    ctx = results(dds, contrast=c("group", "CTX_HFD", "CTX_Chow"))

    dge = list(hpt=as.data.frame(hpt[complete.cases(hpt), ]),
               hippo=as.data.frame(hippo[complete.cases(hippo), ]),
               ctx=as.data.frame(ctx[complete.cases(ctx), ])
    )
    save(dge, file="dge.RData")
} else {
    load("dge.RData")
}

In [15]:
pdf(file="volcano_hpt.pdf", width=10, height=10)
    EnhancedVolcano(dge[['hpt']],
                    lab=rownames(dge[['hpt']]),
                    pCutoff=0.05,
                    FCcutoff=100,
                    axisLabSize=30,
                    x='log2FoldChange',
                    y='pvalue',
                    title=NULL,
                    subtitle=NULL,
                    caption=NULL,
                    ylim=c(0, 8),
                    xlim=c(-8, 8),
                    col=c("grey30", "grey30", "red2", "red2"),
                    legendLabels=c('NS', expression(Log[2]~FC), 'p-value>0.05', expression(p-value~and~log[2]~FC)),
                    legendLabSize=30)
dev.off()

Warning message:
“Removed 2 rows containing missing values (geom_vline).”


pdf 
  2

In [16]:
pdf(file="volcano_hippo.pdf", width=10, height=10)
    EnhancedVolcano(dge[['hippo']],
                    lab=rownames(dge[['hippo']]),
                    pCutoff=0.05,
                    FCcutoff=100,
                    axisLabSize=30,
                    x='log2FoldChange',
                    y='pvalue',
                    title=NULL,
                    subtitle=NULL,
                    caption=NULL,
                    ylim=c(0, 8),
                    xlim=c(-8, 8),
                    col=c("grey30", "grey30", "red2", "red2"),
                    legendLabels=c('NS', expression(Log[2]~FC), 'p-value>0.05', expression(p-value~and~log[2]~FC)),
                    legendLabSize=30)
dev.off()

Warning message:
“Removed 2 rows containing missing values (geom_vline).”


pdf 
  2

In [17]:
pdf(file="volcano_ctx.pdf", width=10, height=10)
    EnhancedVolcano(dge[['ctx']],
                    lab = rownames(dge[['ctx']]),
                    pCutoff=0.05,
                    FCcutoff=100,
                    axisLabSize=30,
                    x='log2FoldChange',
                    y='pvalue',
                    title=NULL,
                    subtitle=NULL,
                    caption=NULL,
                    ylim=c(0, 8),
                    xlim=c(-8, 8),
                    col=c("grey30", "grey30", "red2", "red2"),
                    legendLabels=c('NS', expression(Log[2]~FC), 'p-value>0.05', expression(p-value~and~log[2]~FC)),
                    legendLabSize=30)
dev.off()

Warning message:
“Removed 2 rows containing missing values (geom_vline).”


pdf 
  2

In [18]:
pdf(file="SM/Nr_sig_0.05.pdf", width=10, height=10)
    df = data.frame(Tissue=c("CTX", "HPT", "Hippo"),
                    Nr_significant_genes=c(sum(dge[['ctx']][,5]<0.05, na.rm=T),
                                           sum(dge[['hpt']][,5]<0.05, na.rm=T), 
                                           sum(dge[['hippo']][,5]<0.05, na.rm=T))
    )
    ggplot(df, aes(x=Tissue, y=Nr_significant_genes, fill=Tissue))  +
           geom_bar(stat="identity") + 
            theme(axis.text=element_text(size=20), 
                  axis.title=element_text(size=20), 
                  legend.title=element_text(siz=20),
                  legend.text=element_text(size=20),
                  plot.title=element_text(size=20)) +
           theme(legend.position="none") +
           ggtitle("Number significant genes (p<0.05)") +
           scale_fill_manual(values=c("lightblue", "tan1", "lightgreen"))
dev.off()

pdf 
  2

In [19]:
pdf(file="venn_tran.pdf", width=8, height=8)
    vp = venn.diagram(list(' '=rownames(dge[['hpt']][dge[['hpt']][,5]<0.05,]), 
                           ' '=rownames(dge[['hippo']][dge[['hippo']][,5]<0.05,]),
                           ' '=rownames(dge[['ctx']][dge[['ctx']][,5]<0.05,])),
                     fill=c("orange", "mediumpurple1", "aquamarine2"),
                     filename=NULL,
                     cex=3.5, 
                     col="transparent",
                     cat.cex=0.01,
                     margin=0.01,
                     main.cex=3,
                     main=NULL);
    grid.draw(vp)  
dev.off()

pdf 
  2

## Heatmap marker genes

In [20]:
if(bool_recomp){ 
    vsd1=as.data.frame(vsd[,c(24:35, 1:23)])
    datfin = data.frame(CTX=apply(vsd1 %>% select(starts_with("ctx_")), 1, mean),
                        HPT=apply(vsd1 %>% select(starts_with("hpt_")), 1, mean),
                        Hippo=apply(vsd1 %>% select(starts_with("hippo_")), 1, mean))

    astro=datfin[rownames(datfin)%in%c('Slc1a2', 'Slc1a3', 'Aqp4', 'Gfap', 'Aldh1l1', 'Gja1', 'Gjb6', 'Atp1b2'),]
    neuro=datfin[rownames(datfin)%in%c('Syp', 'Tubb3', 'Snap25', 'Syt1'),]
    mglia=datfin[rownames(datfin)%in%c('Itgam', 'Aif1'),]
    oligo=datfin[rownames(datfin)%in%c('Mog', 'Mag'),]
    endo=datfin[rownames(datfin)%in%c('Slco1c1'),]
    mural=datfin[rownames(datfin)%in%c('Mustn1', 'Pdgfrb', 'Des'),]
    tcyes=datfin[rownames(datfin)%in%c('Crym'),]
    npc=datfin[rownames(datfin)%in%c('Nes', 'Sox2', 'Prom1'),]
    vlmc=datfin[rownames(datfin)%in%c('Col1a1'),]

    astro=astro[order(row.names(astro)),]
    neuro=neuro[order(row.names(neuro)),]
    mglia=mglia[order(row.names(mglia)),]
    oligo=oligo[order(row.names(oligo)),]
    mural=mural[order(row.names(mural)),]
    tcyes=tcyes[order(row.names(tcyes)),]
    mat=rbind(astro, neuro, mglia, oligo, endo, mural) # npc,vlmc,tcyes
    
    write.csv(mat,'celltype_heatmap.csv', row.names=T)
} else{
    mat=read.csv('celltype_heatmap.csv', row.names=1)
}

In [21]:
#Omics=c(rep("transcriptomics", 3))
#Acsa2=rep("Acsa2_pos", 3)
#Tissue=c("cortex", "hypothalamus", "hippocampus")
#dcol=as.data.frame(cbind(Tissue,Acsa2,Omics))
#rownames(dcol)=colnames(mat)
mat=as.matrix(mat)
pheatmap(mat, 
         scale="column",
         show_rownames=F, 
         show_colnames=F,
         color=my_palette, 
         cluster_cols=F,
         cluster_rows=F, 
         gaps_col=3,
         gaps_row=c(8,12,14,16,17,20),#(11,15,17,19,20,23), 
         fontsize=18,
         filename="heatmap_ACSA2.pdf",
         legend=F,
         width=3.3, 
         height=8
) 

## Heatmap DGE

#### HPT

In [22]:
vsd_hpt = vsd[rownames(vsd)%in%rownames(dge[['hpt']][dge[['hpt']][,5]<0.05,]),] 
pheatmap(vsd_hpt[,1:11], 
         scale="row",
         show_rownames=FALSE,
         show_colnames=FALSE,
         color=my_palette, 
         cluster_cols=F,
         legend=FALSE,
         fontsize=15,
         filename="heatmap_hpt.pdf",
         width=5,
         height=8,
         breaks=seq(-2, 2, length.out=255)) 

#### Hippo

In [23]:
vsd_hip = vsd[rownames(vsd)%in%rownames(dge[['hippo']][dge[['hippo']][,5]<0.05,]),] 
pheatmap(vsd_hip[,12:23], 
         scale="row", 
         show_rownames=FALSE,
         show_colnames=FALSE,
         color=my_palette,
         cluster_cols=F, 
         legend=FALSE,
         fontsize=15, 
         filename="heatmap_hippo.pdf",
         width=5, 
         height=8,
         breaks=seq(-2, 2, length.out=255)
) 

#### CTX

In [24]:
vsd_ctx = vsd[rownames(vsd)%in%rownames(dge[['ctx']][dge[['ctx']][,5]<0.05,]),] 
pheatmap(vsd_ctx[,24:35],
         scale="row",
         show_rownames=FALSE,
         show_colnames=FALSE,
         color=my_palette, 
         cluster_cols=F,
         legend=F,
         fontsize=15,
         filename="heatmap_ctx.pdf",
         width=5, 
         height=8,
         breaks=seq(-2, 2, length.out=255)
)

## GO - enrichment

In [25]:
if(bool_recomp){ 
    sg.ctx=bitr(rownames(vsd_ctx), fromType="SYMBOL", toType="ENTREZID", OrgDb=org.Mm.eg.db)
    go.ctx=enrichGO(sg.ctx[,2], 'org.Mm.eg.db', ont="BP", pvalueCutoff=0.1)

    sg.hpt=bitr(rownames(vsd_hpt), fromType="SYMBOL", toType="ENTREZID", OrgDb=org.Mm.eg.db)
    go.hpt=enrichGO(sg.hpt[,2], 'org.Mm.eg.db', ont="BP", pvalueCutoff=0.1)

    sg.hip=bitr(rownames(vsd_hip), fromType="SYMBOL", toType="ENTREZID", OrgDb=org.Mm.eg.db)
    go.hip=enrichGO(sg.hip[,2], 'org.Mm.eg.db', ont="BP", pvalueCutoff=0.1)

    go = c("insulin receptor signaling pathway",
            "response to insulin",
            "TOR signaling",
            "pyruvate metabolic process",
            "intracellular lipid transport",
            "fatty acid metabolic process",
            "fatty acid oxidation",
            "response to reactive oxygen species",
            "response to endoplasmic reticulum stress",
            "regulation of mitotic cell cycle",
            "transforming growth factor beta receptor signaling pathway",
            "I-kappaB kinase/NF-kappaB signaling",
            "regulation of inflammatory response",
            "regulation of cell junction assembly",
            "cell-matrix adhesion",
            "regulation of cytosolic calcium ion concentration",
            "vesicle budding from membrane",
            "axon ensheathment",
            "maintenance of synapse structure",
            "regulation of synapse structure or activity",
            "synaptic transmission, GABAergic",
            "AMPA glutamate receptor clustering",
            "positive regulation of blood vessel endothelial cell migration",
            "sprouting angiogenesis",
            "regulation of angiogenesis"
    )
    
    enr_ctx.go=go.ctx@result[go.ctx@result[,6]<0.05, c(2, 5, 6, 8, 9)]
    enr_ctx.go$Tissue='CTX'
    enr_hpt.go=go.hpt@result[go.hpt@result[,6]<0.05, c(2, 5, 6, 8, 9)]
    enr_hpt.go$Tissue='HPT'
    enr_hip.go=go.hip@result[go.hip@result[,6]<0.05, c(2, 5, 6, 8, 9)]
    enr_hip.go$Tissue='Hippo'
    
    all = rbind(enr_ctx.go, enr_hpt.go, enr_hip.go)
    #write.csv(all, file="GO_transcriptome.csv")
    all = all[all$Description%in%go,]
    options(repr.plot.width=20, repr.plot.height=15)
    colnames(all) = c('Description', 'pvalue', 'p.adjust', 'geneID', 'count', 'Tissue')

    all=rbind(enr_ctx.go, enr_hpt.go, enr_hip.go)
    all1=all[all$Description%in%go,]

    sig.gene.hpt.all = bitr(rownames(dge[['hpt']]), fromType="SYMBOL", toType="ENTREZID", OrgDb=org.Mm.eg.db)
    sig.gene.ctx.all = bitr(rownames(dge[['ctx']]), fromType="SYMBOL", toType="ENTREZID", OrgDb=org.Mm.eg.db)
    sig.gene.hip.all = bitr(rownames(dge[['hippo']]), fromType="SYMBOL", toType="ENTREZID", OrgDb=org.Mm.eg.db)
    
    # CTX
    df_total.ctx=data.frame()
    for(i in 1:nrow(all1)){   
        p=as.data.frame(dge[['ctx']][which(rownames(dge[['ctx']])%in%
                         sig.gene.ctx.all[sig.gene.ctx.all[,2]%in%unlist(strsplit(all1$geneID[i],"/")),1]), c(2,5)])
        p$pathway=all1$Description[i]
        p$padjPath=all1$p.adjust[i]
        p$count=all1$Count[i]
        df_total.ctx=rbind(df_total.ctx, p)    
    }
    df_total.ctx$significance=ifelse(df_total.ctx$pvalue<0.05, "significant", "not significant")

    # HPT
    df_total.hpt = data.frame()
    for(i in 1:nrow(all1)){
        p=as.data.frame(dge[['hpt']][which(rownames(dge[['hpt']])%in%
                         sig.gene.hpt.all[sig.gene.hpt.all[,2]%in%unlist(strsplit(all1$geneID[i],"/")),1]),c(2,5)])
        p$pathway=all1$Description[i]
        p$padjPath=all1$p.adjust[i]
        p$count=all1$Count[i]
        df_total.hpt=rbind(df_total.hpt, p)    
    } 
    df_total.hpt$col=ifelse(df_total.hpt$pvalue<0.05, "red", "grey")
    
    # Hippo
    df_total.hip=data.frame()
    for(i in 1:nrow(all1)){
        p=as.data.frame(dge[['hippo']][which(rownames(dge[['hippo']])%in%
                         sig.gene.hip.all[sig.gene.hip.all[,2]%in%unlist(strsplit(all1$geneID[i],"/")),1]), c(2,5)])
        p$pathway=all1$Description[i]
        p$padjPath=all1$p.adjust[i]
        p$count=all1$Count[i]
        df_total.hip=rbind(df_total.hip, p)    
    }    
    df_total.hip$col=ifelse(df_total.hip$pvalue<0.05, "red", "grey")

    gsea = list(all=all, ctx=df_total.ctx, hpt=df_total.hpt, hippo=df_total.hip, go=go)
    save(gsea, file="gsea.RData")
} else {
    load("gsea.RData")
}

In [26]:
pdf(file="enrich_path.pdf", width=15, height=15)
    ggplot(gsea[['all']],   
           aes(Tissue, y=factor(Description, level=gsea[['go']][length(gsea[['go']]):1]), col=pvalue, size=Count)) + 
           geom_point() + 
           scale_colour_gradient(low="red", high="blue") +
           scale_size_continuous(range=c(3, 10)) +
           xlab("") + 
           ylab("") + 
           theme(axis.text.x=element_blank()) + theme(legend.position="none") +
           theme(text=element_text(size=40))
dev.off()

pdf 
  2

#### CTX

In [27]:
pdf(file="enrichdot_ctx.pdf", width=7, height=15)
    ggplot(gsea[['ctx']], aes(log2FoldChange, y=factor(pathway, level=gsea[['go']][length(gsea[['go']]):1]))) + 
           geom_point(aes(colour=significance, size=-log10(pvalue)), alpha=0.5) + 
           scale_size_continuous(range=c(0,8)) +
           scale_color_manual(values=c("gray85", "red")) +
           geom_vline(xintercept=0) +
           theme(axis.title=element_text(size=20),
                 title=element_text(size=20), 
                 text=element_text(size=35)) + #guides(color = guide_legend(override.aes = list(size=5) ) ) +
           theme(legend.position="none") + 
           theme(axis.text.y=element_blank()) +
           ylab("") + xlab("") + xlim(-2.68, 3.82)
dev.off() 

Warning message:
“Removed 12 rows containing missing values (geom_point).”


pdf 
  2

#### HPT

In [28]:
pdf(file="enrichdot_hpt.pdf", width=7, height=15)

    ggplot(gsea[['hpt']], aes(log2FoldChange, y=factor(pathway, level=gsea[['go']][length(gsea[['go']]):1]))) + 
           geom_point(aes(colour=col, size=-log10(pvalue)), alpha=0.5) + 
           scale_size_continuous(range=c(0, 8)) +
           scale_color_manual(values=c("gray85", "red")) + 
           geom_vline(xintercept=0) +
           theme(axis.title=element_text(size=20),
                 title=element_text(size=20), 
                 text=element_text(size=35)) +
           theme(legend.position="none") + 
           theme(axis.text.y=element_blank()) +
           ylab("") + xlab("") + xlim(-2.68,3.82)
dev.off()

Warning message:
“Removed 5 rows containing missing values (geom_point).”


pdf 
  2

#### Hippo

In [29]:
pdf(file = "enrichdot_hippo.pdf", width=7, height=15)

    ggplot(gsea[['hippo']], aes(log2FoldChange, y=factor(pathway, level=gsea[['go']][length(gsea[['go']]):1]))) + 
           geom_point(aes(colour=col, size=-log10(pvalue)), alpha=0.5) + 
           scale_color_manual(values=c("gray85", "red")) + 
           scale_size_continuous(range = c(0, 6)) + 
           geom_vline(xintercept = 0) +
           theme(axis.title=element_text(size=20),
                 title=element_text(size=20), 
                 text = element_text(size=35)) +
           theme(legend.position="none") + 
           theme(axis.text.y=element_blank()) +
           ylab("") + xlab("") + xlim(-2.68, 3.82)
dev.off()

Warning message:
“Removed 15 rows containing missing values (geom_point).”


pdf 
  2